## Athena Data Ingestion

In this section, we query and retrieve data from Amazon Athena for further processing. Athena allows us to perform SQL-based queries on structured data stored in Amazon S3, enabling efficient data exploration and analysis without the need for complex ETL pipelines.  

In [1]:
!pip install pyathena --quiet

In [2]:
# AWS Imports
import boto3
from botocore.client import ClientError
import sagemaker
from pyathena import connect
import awswrangler as wr

# Data Transformation Imports
import pandas as pd
from io import StringIO

# Misc Imports
from IPython.display import display, HTML

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
# Create a SageMaker session object, which is used to manage interactions with SageMaker resources.
sess = sagemaker.Session()

# Retrieve the default Amazon S3 bucket associated with the SageMaker session.
bucket = sess.default_bucket()

# Get the IAM role associated with the current SageMaker notebook or environment.
role = sagemaker.get_execution_role()

# Get the AWS region name for the current session.
region = boto3.Session().region_name

# Retrieve the AWS account ID of the caller using the Security Token Service (STS) client.
account_id = boto3.client("sts").get_caller_identity().get("Account")

# Create a Boto3 client for the SageMaker service, specifying the AWS region.
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

# Create an S3 client
s3 = boto3.client('s3')

# create athena client 
athena_client = boto3.client('athena', region_name=region)

## Sagemaker Details and Variable Init

In [4]:
# Initializing variables for reproducibility
FILE_NAME="remaining_data.csv"
DATA_SOURCE="db_source"
DATA_FOLDER =f"s3://{bucket}/aai-540-group-3-final-project/data/"
FILE_LOCATION=f"{DATA_FOLDER}{DATA_SOURCE}/{FILE_NAME}"
DATA_PATH = f"{DATA_FOLDER}{DATA_SOURCE}/"
DATABASE = "retainAI"
PROD_DIR = f"s3://{bucket}/athena/prod"
STAGE_DIR = f"s3://{bucket}/athena/staging"
EMPLOYEE_TABLE = "employee_table"

# Making sure all variables are correct
print(f"File location with all the data: {FILE_LOCATION}")
print(f"Data Path for database creation: {DATA_PATH}")
print(f"Production and Staging Database Directories: {PROD_DIR}, {STAGE_DIR}")
print(f"Database Name, Training Table and Testing Table: {DATABASE}, {EMPLOYEE_TABLE}")

File location with all the data: s3://sagemaker-us-east-1-489552815489/aai-540-group-3-final-project/data/db_source/remaining_data.csv
Data Path for database creation: s3://sagemaker-us-east-1-489552815489/aai-540-group-3-final-project/data/db_source/
Production and Staging Database Directories: s3://sagemaker-us-east-1-489552815489/athena/prod, s3://sagemaker-us-east-1-489552815489/athena/staging
Database Name, Training Table and Testing Table: retainAI, employee_table


In [5]:
# check what is in DATA_FOLDER
!aws s3 ls $DATA_FOLDER --recursive

2025-02-22 23:11:16    2616747 aai-540-group-3-final-project/data/db_source/remaining_data.csv
2025-02-22 23:11:21    9550295 aai-540-group-3-final-project/data/eda/data.csv
2025-02-22 23:11:19    1744315 aai-540-group-3-final-project/data/holdout/holdout.csv


## Creating Athena Schema

In [7]:
# Establish a connection to the AWS Athena service, specifying the region and an S3 staging directory
# where query results will be stored.
conn = connect(region_name=region, s3_staging_dir=STAGE_DIR)

In [8]:
# Delete Athena db if exists 
db_drop_statement = f"DROP DATABASE IF EXISTS {DATABASE} CASCADE;"

# Run the DROP DATABASE query
response = athena_client.start_query_execution(
    QueryString=db_drop_statement,
    QueryExecutionContext={
        'Database': DATABASE  # Specify your database here
    },
    ResultConfiguration={
        'OutputLocation': STAGE_DIR
    }
)

print(response)

{'QueryExecutionId': 'fd509319-1d0e-47fe-b6d1-aab7c3e8f69a', 'ResponseMetadata': {'RequestId': 'cd7f90f8-c908-4527-89c1-3316bb1843b3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 22 Feb 2025 23:12:24 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '59', 'connection': 'keep-alive', 'x-amzn-requestid': 'cd7f90f8-c908-4527-89c1-3316bb1843b3'}, 'RetryAttempts': 0}}


In [9]:
# Define a SQL statement to create a database in Athena if it doesn't already exist.
# The database name is dynamically determined by the variable `DATABASE`.
db_create_statement = f"CREATE DATABASE IF NOT EXISTS {DATABASE}"

# Execute the SQL statement using the established connection and Pandas, 
# which sends the query to Athena and ensures the database is created.
pd.read_sql(db_create_statement, conn)

/tmp/ipykernel_336/3777991661.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(db_create_statement, conn)


""


In [10]:
# Define a SQL statement to list all existing databases in the AWS Athena service.
show_db_statement = "SHOW DATABASES"

# Execute the SQL query using the established connection, and store the results in a Pandas DataFrame.
df_show = pd.read_sql(show_db_statement, conn)

# Display the first 5 rows of the DataFrame to view the list of databases.
df_show.head(5)

/tmp/ipykernel_336/1619648943.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(show_db_statement, conn)


,database_name
0,default
1,retainai


## Registering Data with Athena

### Creating Employee Data Table

In [11]:
CREATE_STATEMENT = """
CREATE EXTERNAL TABLE IF NOT EXISTS {}.{} (
    employee_id INT,
    age INT,
    gender STRING,
    years_at_company INT,
    job_role STRING,
    monthly_income INT,
    work_life_balance STRING,
    job_satisfaction STRING,
    performance_rating STRING,
    number_of_promotions INT,
    distance_from_home INT,
    education_level STRING,
    marital_status STRING,
    number_of_dependents INT,
    job_level STRING,
    company_size STRING,
    company_tenure INT,
    remote_work STRING,
    leadership_opportunities STRING,
    innovation_opportunities STRING,
    company_reputation STRING,
    employee_recognition STRING,
    attrition STRING
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\\n'
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')
""".format(DATABASE, EMPLOYEE_TABLE, DATA_PATH)

# Ensuring everything is formatted correctly
print(CREATE_STATEMENT)


CREATE EXTERNAL TABLE IF NOT EXISTS retainAI.employee_table (
    employee_id INT,
    age INT,
    gender STRING,
    years_at_company INT,
    job_role STRING,
    monthly_income INT,
    work_life_balance STRING,
    job_satisfaction STRING,
    performance_rating STRING,
    number_of_promotions INT,
    distance_from_home INT,
    education_level STRING,
    marital_status STRING,
    number_of_dependents INT,
    job_level STRING,
    company_size STRING,
    company_tenure INT,
    remote_work STRING,
    leadership_opportunities STRING,
    innovation_opportunities STRING,
    company_reputation STRING,
    employee_recognition STRING,
    attrition STRING
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
LOCATION 's3://sagemaker-us-east-1-489552815489/aai-540-group-3-final-project/data/db_source/'
TBLPROPERTIES ('skip.header.line.count'='1')



In [12]:
# Execute the SQL query defined in CREATE_STATEMENT to create the employee data table in Athena. 
pd.read_sql(CREATE_STATEMENT, conn)

/tmp/ipykernel_336/75521144.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(CREATE_STATEMENT, conn)


""


### Check Tables and Ensure Correctness

If everything runs correctly, you should see tab_name = employee_table

In [13]:
check_table_statement = "SHOW TABLES in {}".format(DATABASE)

df_show = pd.read_sql(check_table_statement, conn)
df_show

/tmp/ipykernel_336/457795480.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(check_table_statement, conn)


,tab_name
0,employee_table


In [14]:
# If everything runs as expected, you should get a return value of 74,498 instances in our table
count_records = f"""SELECT COUNT(*) FROM {DATABASE}.{EMPLOYEE_TABLE}"""
count_show = pd.read_sql(count_records,conn)
count_show

/tmp/ipykernel_336/2012675176.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  count_show = pd.read_sql(count_records,conn)


,_col0
0,44699


## Release Resources

In [2]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [14]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>